# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [170]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
from create_tables import *

In [171]:
# connect to default database
conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=student")
conn.set_session(autocommit=True)
cur = conn.cursor()
print(conn)

<connection object at 0x7f6c720ee508; dsn: 'user=student password=xxx dbname=studentdb host=127.0.0.1', closed: 0>


In [172]:
#conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
#conn.set_session(autocommit=True)
#cur = conn.cursor()
#print(conn)

In [169]:
#CLOSE CONNECTION
cur.close()
conn.close()
print(conn)

<connection object at 0x7f6c720ee048; dsn: 'user=student password=xxx dbname=sparkifydb host=127.0.0.1', closed: 1>


In [173]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [174]:
song_files = get_files('data/song_data')

In [175]:
type(song_files)
len(song_files)
song_files[0]

'/home/workspace/data/song_data/A/B/C/TRABCIX128F4265903.json'

In [176]:
len(song_files)

71

In [177]:
filepath = song_files[0]

In [178]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNF6401187FB57032,40.79086,"New York, NY [Manhattan]",-73.96644,Sophie B. Hawkins,305.162,1,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,1994


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [179]:
#song_id int, title varchar, artist_id int, year int, duration int 
#song_data = df[.values.tolist()[0]]
song_data = df[['song_id','title','artist_id','year','duration']].values.tolist()[0]
song_data

['SONWXQJ12A8C134D94',
 'The Ballad Of Sleeping Beauty',
 'ARNF6401187FB57032',
 1994,
 305.162]

In [180]:
df.columns

Index(['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year'],
      dtype='object')

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [181]:
song_table_create

'\nCREATE TABLE IF NOT EXISTS songs(\nsong_id varchar not null primary key, \ntitle varchar, \nartist_id varchar, \nyear int, \nduration float8\n)\n'

In [182]:
#song_table_drop

In [183]:
##RUN IF DROP IS NEEDED
#cur.execute(song_table_drop)
#conn.commit()

In [184]:
##CREATE TABLE
cur.execute(song_table_create)
conn.commit()

In [185]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [186]:
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNF6401187FB57032,40.79086,"New York, NY [Manhattan]",-73.96644,Sophie B. Hawkins,305.162,1,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,1994


In [187]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values.tolist()[0]
artist_data

['ARNF6401187FB57032',
 'Sophie B. Hawkins',
 'New York, NY [Manhattan]',
 40.79086,
 -73.96644]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [188]:
artist_table_create

'\nCREATE TABLE IF NOT EXISTS artists(\nartist_id varchar not null primary key, \nname varchar, \nlocation varchar, \nlattitude float8, \nlongitude float8\n)\n'

In [189]:
##RUN IF DROP IS NEEDED
#cur.execute(artist_table_drop)
#conn.commit()

In [190]:
##CREATE TABLE
cur.execute(artist_table_create)
conn.commit()

In [191]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [192]:
log_files = get_files('data/log_data')

In [193]:
filepath = log_files[0]

In [194]:
df = pd.read_json(filepath, lines=True)
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [195]:
df = df[df['page'] == 'NextSong']
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
1,Beastie Boys,Logged In,Harper,M,2,Barrett,161.56689,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Lighten Up,200,1541722186796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


In [196]:
t = df.copy()
t['ts'] = pd.to_datetime(df['ts'])
t.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1970-01-01 00:25:41.721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


In [197]:
time_data = t['ts'].tolist()
type(time_data)

list

In [198]:
#Extract the timestamp, hour, day, week of year, month, year, and weekday
timestamp = []
time_data_hour = []
time_data_day = []
time_data_week = []
time_data_month = []
time_data_year = []
time_data_weekday = []

for timestamp in time_data:
    timestamp = timestamp
    time_data_hour.append(timestamp.hour)
    time_data_day.append(timestamp.day)
    time_data_week.append(timestamp.week)
    time_data_month.append(timestamp.month)
    time_data_year.append(timestamp.year)
    time_data_weekday.append(timestamp.weekday())
    #time_data[0],time_data_hour,time_data_day,time_data[0].week,time_data_month,time_data_year,time_data_weekday
#column_labels = ()

In [199]:
column_labels = pd.DataFrame({
    'start_time': timestamp,
    'hour':time_data_hour,
    'day':time_data_day,
    'week':time_data_week,
    'month':time_data_month,
    'year':time_data_year,
    'weekday':time_data_weekday
})
column_labels.head()

,start_time,hour,day,week,month,year,weekday
0,1970-01-01 00:25:41.805849796,0,1,1,1,1970,3
1,1970-01-01 00:25:41.805849796,0,1,1,1,1970,3
2,1970-01-01 00:25:41.805849796,0,1,1,1,1970,3
3,1970-01-01 00:25:41.805849796,0,1,1,1,1970,3
4,1970-01-01 00:25:41.805849796,0,1,1,1,1970,3


In [200]:
time_df = column_labels.copy()
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,1970-01-01 00:25:41.805849796,0,1,1,1,1970,3
1,1970-01-01 00:25:41.805849796,0,1,1,1,1970,3
2,1970-01-01 00:25:41.805849796,0,1,1,1,1970,3
3,1970-01-01 00:25:41.805849796,0,1,1,1,1970,3
4,1970-01-01 00:25:41.805849796,0,1,1,1,1970,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [201]:
time_table_create

'\nCREATE TABLE IF NOT EXISTS time(\ntime_id serial not null primary key,\nstart_time timestamp, \nhour int, \nday int, \nweek int, \nmonth int, \nyear int, \nweekday int\n)\n'

In [202]:
##CREATE TABLE
cur.execute(time_table_create)
conn.commit()

In [203]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [204]:
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


In [205]:
user_df = df[['userId','firstName','lastName','gender','level']]
user_df.head(3)

,userId,firstName,lastName,gender,level
0,42,Harper,Barrett,M,paid
1,42,Harper,Barrett,M,paid
2,42,Harper,Barrett,M,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [206]:
user_table_create

'\nCREATE TABLE IF NOT EXISTS users(\nuser_id int not null primary key, \nfirst_name varchar, \nlast_name varchar, \ngender varchar, \nlevel varchar\n)\n'

In [207]:
##CREATE TABLE
cur.execute(user_table_create)
conn.commit()

In [208]:
for i, row in user_df.iterrows():
    #print(row.values)
    #print(list(row))
    try:
        cur.execute(user_table_insert, list(row))
        conn.commit()
    except:
        continue

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [209]:
songplay_table_create

'\nCREATE TABLE IF NOT EXISTS songplays(\nsongplay_id serial not null primary key, \nstart_time timestamp, \nuser_id varchar, \nlevel varchar, \nsong_id varchar, \nartist_id varchar, \nsession_id int, \nlocation varchar, \nuser_agent varchar\n)\n'

In [210]:
##CREATE TABLE
cur.execute(songplay_table_create)
conn.commit()

In [211]:
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


In [212]:
song_select_test = ("""SELECT song_id, songs.artist_id FROM songs \
JOIN artists ON songs.artist_id = artists.artist_id \
WHERE songs.title = %s AND artists.name = %s AND songs.duration = %s\
;""")
type(song_select_test),song_select_test

(str,
 'SELECT song_id, songs.artist_id FROM songs JOIN artists ON songs.artist_id = artists.artist_id WHERE songs.title = %s AND artists.name = %s AND songs.duration = %s;')

In [251]:
songplay_table_insert_test = """INSERT INTO songplays(song_id, artist_id)"""
songplay_table_insert_test, type(songplay_table_insert_test)

('INSERT INTO songplays(song_id, artist_id);', str)

In [255]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    song = row.song
    artist = row.artist
    length = row.length
    #print(song, artist, length)
    cur.execute(song_select_test, (song, artist, length))
    results = cur.fetchone()
    print(song, artist, length,results)
    #print(results[0],results[2])
    #print(list(results))
    
    if results:
        songid, artistid = results
        songid, artistid = [results[0],results[2]]
    else:
        songid, artistid = None, None

    #insert songplay record
    songplay_data = results
    #print(songplay_data)
    #cur.execute(songplay_table_insert_test, songplay_data)
    #conn.commit()

Supermassive Black Hole (Twilight Soundtrack Version) Muse 209.50159 None
Lighten Up Beastie Boys 161.56689 None
Pienso En Ti Shakira 145.84118 None
Amor Prohibido Selena 172.66893 None
Day 'N' Nite Kid Cudi Vs Crookers 162.97751 None
Black Masks & Gasoline Rise Against 179.59138 None
I'm Like A Lawyer With The Way I'm Always Trying To Get You Off (Me & You) Fall Out Boy 211.69587 None
Staple It Together Jack Johnson 196.12689 None
Slow Jamz (Feat. Kanye West & Jamie Foxx) (Edited Album Version) Twista feat. Kayne West & Jamie Foxx 212.55791 None
Hey_ Soul Sister Train 216.76363 None
Rehab Rihanna 293.82485 None
Van Helsing Boombox Man Man 222.45832 None
The Stallion Ween 276.13995 None
Fireflies Charttraxx Karaoke 225.17506 None
I Want I Want Digitalism 208.24771 None
Railway Dispatch 247.50975 None
Clark Gable (Album) Postal Service 294.47791 None
Phaeton Patricia Barber 317.83138 None
Ucci Ucci feat. Nicki Minaj & The Chopper City Boyz Enur 203.4673 None
Choke Hybrid 379.61098 None


UnicodeEncodeError: 'ascii' codec can't encode characters in position 1-4: ordinal not in range(128)

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [257]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.